In [ ]:
! pip install numpy
! pip install pandas
! pip install matplotlib
! pip install Ipython

In [ ]:
#Import Relevant Modules
import re
from datetime import datetime, date
from time import sleep
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates

In [ ]:
#Retrive today's date
date = date.today()

In [ ]:
def getPMONData(filename):
    '''
    Extract PMON Temperature data from txt file

    @param:
        filename -> str
    @return:
        list with data from each PMON
    '''
    
    data = [] 
    pmon_pattern = re.compile(r'\[(\d{2}:\d{2}:\d{2})\].*?PMON\s+temps:\s*(\{[^}]+\})', re.I)
    with open(filename, "r") as f:
        for line in f:
            p = pmon_pattern.search(line)
            if p:
                time_str = p.group(1)
                log_time = datetime.strptime(time_str, "%H:%M:%S").time()
                
                pmon_dict = eval(p.group(2))
                pmon_1 = pmon_dict[1]
                pmon_2 = pmon_dict[2]
                pmon_3 = pmon_dict[3]
                pmon_4 = pmon_dict[4]
                pmon_5 = pmon_dict[5]
                pmon_6 = pmon_dict[6]

                data.append((log_time, pmon_1, pmon_2, pmon_3, pmon_4, pmon_5, pmon_6))
    return data

In [ ]:
def createPMONCSV(pmon_data):
    '''

    @param:
    pmon_data: list of data -> list
    '''
    with open(f'pmon_log-' + str(date) + '.csv', 'w', newline='') as out:
        writer = csv.writer(out)
        writer.writerow(['Time', 'PMON 1', 'PMON_2', 'PMON_3', 'PMON_4', 'PMON_5', 'PMON_6'])   # header
        writer.writerows(pmon_data)

    print(f'{len(pmon_data)} rows written to pmon_log-' + str(date) + '.csv')

In [ ]:
def getPMONGraph(csv_file, start_time = '00:00:01', end_time = '23:59:59'):
    '''
    Displays PMON Graph

    args:
        csv_file : File name -> str
        start_time : -> str
    
    '''
    pmon_result = pd.read_csv(csv_file, parse_dates=['Time'])
    pmon_result['timestamp'] = pd.to_datetime(pmon_result['Time'], format='%H:%M:%S')
    pmon_result = pmon_result.to_numpy()
    time_DNE = False

    time = pd.to_datetime(pmon_result[:, 0], format='%H:%M:%S')
    pmon_data = {}

    try:
        start_index = time.tolist().index(str(start_time))
        end_index = time.tolist().index(str(end_time))
    except ValueError:
        print("Timeframe does not exist")
        time_DNE = True

    ports = {1:'PMON_1', 2:'PMON_2', 3:'PMON_3', 4:'PMON_4', 5:'PMON_5', 6:'PMON_6'}

    for idx, value in ports.items():
        if time_DNE == True:
            pmon_data[value] = pmon_result[:, idx].tolist()
        elif time_DNE == False:
            pmon_data[value] = pmon_result[start_index:end_index, idx].tolist()
            
    if time_DNE == False:
        time = time[start_index:end_index]

    #Method 1 - Completely remove the disconnected value (data will be shorter than the rest)
    #for key in ports.values():
    #    pmon_data[key] = [float(x) for x in pmon_data[key] if x not in ('DISCONNECTED','SENSOR_ERROR')]

    #Method 2 - fill data points that are disconnected with previous value
    for key in ports.values():
        if pmon_data[key][0] in ['DISCONNECTED','SENSOR_ERROR', 'None']:
            pmon_data[key] = [float(x) for x in pmon_data[key] if x not in ('DISCONNECTED','SENSOR_ERROR', 'None')]
        else:
            for i in range(len(pmon_data[key])):
                if pmon_data[key][i] in ['DISCONNECTED','SENSOR_ERROR','None']:
                    pmon_data[key][i] = pmon_data[key][i - 1]
            pmon_data[key] = [float(x) for x in pmon_data[key]]


    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot()
    
    for key in ports.values():
        if pmon_data[key]:
            print(f'Average Temperature for {key}: {np.mean(pmon_data[key])}')
        else:
            print(f'No data from {key}')
        ax.plot(time, pmon_data[key], label=key)
    
    ax.set_xlabel('Time Elapsed (s)')
    ax.set_ylabel('Temperature (°C)')
    ax.set_title('Temperature Graph')
    ax.grid()
    ax.legend(loc='upper right')

    # Set number of X axis tick marks
    ax.xaxis.set_major_locator(MaxNLocator(nbins=20))

    # Format x-axis to show only the time (HH:MM)
    time_format = mdates.DateFormatter('%H:%M:%S')
    ax.xaxis.set_major_formatter(time_format)

    # Rotate x axis labels
    ax.figure.autofmt_xdate()

    

    plt.show()


In [ ]:
# Log file location on laptop: 'C:/Users/Experiment/EBEAM_dashboard/EBEAM-Dashboard-Logs/'

createPMONCSV(getPMONData('Data samples/log_export_2025-06-09_17-10-43.txt'))
getPMONGraph(f'pmon_log-' + str(date) + '.csv')

In [ ]:
os.path.abspath("C:/Users/Experiment/EBEAM_dashboard/EBEAM-Dashboard-Logs/log_2025-06-11_09-19-22.txt")

while 1 :
    createPMONCSV(getPMONData('C:/Users/Experiment/EBEAM_dashboard/EBEAM-Dashboard-Logs/log_2025-06-11_09-19-22.txt'))
    getPMONGraph(f'pmon_log-' + str(date) + '.csv')  
    sleep(10)
    clear_output(wait=True)